# In-Context Learning Approach 

In [1]:
import requests
import json
import pandas as pd
from typing import List, Tuple
import math
import requests
import json
import pandas as pd
from typing import List, Tuple
import math
import gc
import os
import pandas as pd
from tqdm import tqdm  # for progress tracking
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

url = "http://localhost:11434/api/chat"

In [3]:
from langchain_ollama import ChatOllama


llm = ChatOllama(
    model = "qwen2.5:14b" #qwen2 (7B), ollama run qwen2.5:7b, qwen2.5:14b, mistral-nemo, phi3.5
)

In [4]:
pd.options.display.max_colwidth = 1000

In [2]:
dir_path = 'combined_500ms_whisper_diarization_stable_ts_aligned'

# Initialize an empty dictionary to store the data
aligned_dict = {}

# Iterate through files in the directory
for file_name in os.listdir(dir_path):
    # Check if the file is a CSV
    if file_name.endswith('.csv'):
        # Create the full file path
        file_path = os.path.join(dir_path, file_name)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Use the file name without the extension as the dictionary key
        key = os.path.splitext(file_name)[0]
        
        # Store the DataFrame in the dictionary
        aligned_dict[key] = df

keys = list(aligned_dict.keys())

# Group keys by their first three digits
grouped_keys = {}
for key in keys:
    prefix = key[:3]
    if prefix not in grouped_keys:
        grouped_keys[prefix] = []
    grouped_keys[prefix].append(key)

# Get list of prefixes
prefixes = list(grouped_keys.keys())

test_prefixes =  ['051', '134', '251', '027', '129', '038', '108', '107', '252']

all_train_prefixes = list(set(prefixes) - set(test_prefixes))

train_prefixes, val_prefixes = train_test_split(all_train_prefixes, test_size=0.10, random_state=99)

train_keys = [key for key in keys if key[:3] in train_prefixes]
val_keys = [key for key in keys if key[:3] in val_prefixes]
test_keys = [key for key in keys if key[:3] in test_prefixes]

In [3]:
train_dfs = [aligned_dict[key][['duration',"speaker", "label", "transcript"]] for key in train_keys]
val_dfs = [aligned_dict[key][['duration',"speaker", "label",  "transcript"]] for key in val_keys]
test_dfs = [aligned_dict[key][['duration',"speaker", "label", "transcript"]] for key in test_keys]

train_df = pd.concat(train_dfs, ignore_index=True)
val_df = pd.concat(val_dfs, ignore_index=True)
test_df = pd.concat(test_dfs, ignore_index=True)

In [4]:
train_df = train_df[train_df["speaker"] != "Student"]
val_df = val_df[val_df["speaker"] != "Student"]
test_df = test_df[test_df["speaker"] != "Student"]

train_df = train_df[train_df["label"] != "UNI"]
val_df = val_df[val_df["label"] != "UNI"]
test_df = test_df[test_df["label"] != "UNI"]

train_df = train_df[train_df["duration"] > 0.5]
val_df = val_df[val_df["duration"] > 0.5]
test_df = test_df[test_df["duration"] >0.5]

In [5]:
# Remove UNI from labels for all datasets
train_df['label'] = train_df['label'].apply(lambda x: ' '.join([label for label in x.split() if label not in ['UNI', 'SIL']]))
val_df['label'] = val_df['label'].apply(lambda x: ' '.join([label for label in x.split() if label not in ['UNI', 'SIL']]))
test_df['label'] = test_df['label'].apply(lambda x: ' '.join([label for label in x.split() if label not in ['UNI', 'SIL']]))

In [6]:
import json
import gc
from tqdm import tqdm

def process_batch(llm, batch_rows):
    """Process a batch of transcript rows and return the results"""
    # Format all rows in batch for the prompt
    formatted_rows = "\n\n".join([
        f"Interaction {i+1}:\nSpeaker: {row['speaker']}\nDuration: {row['duration']}\nTranscript: {row['transcript']}"
        for i, row in enumerate(batch_rows)
    ])
    
    messages = [
        ("system", f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an education researcher analyzing classroom interactions and teaching practices.

TASK: Extract and label classroom practices from multiple transcript texts using standardized codes.

OUTPUT FORMAT: [{{
    "interaction_number": "Number of the interaction in batch",
    "text": "Original utterance or interaction",
    "codes": ["List of applicable codes"],
    "speaker": "Speaker identifier",
    "duration": "Duration value"
}}]

Key Rules:
1. Code Application:
   - Apply all relevant codes to each utterance
   - Multiple codes may apply to a single utterance
   - Process ALL interactions in the input
   - Use exact code values from the provided list:
     - IST: Instructional Talk
     - ST: Social Talk
     - aOTR: Academic Opportunity to Respond
     - sOTR: Social Opportunity to Respond
     - REP: Reprimand
     - RED: Redirection
     - BSP: Behavior Specific Praise
     - GPRS: General Praise
     - aAFF: Academic Affirmation
     - aCORR: Academic Corrective
     - NEU: Neutral

2. Text Processing:
   - Process each interaction separately
   - Preserve complete utterances for context
   - Include speaker and duration information
   - Keep related interactions together
   - Return results for ALL interactions in the batch

Return only valid JSON without explanations.<|eot_id|>"""),
        ("user", f"<|start_header_id|>user<|end_header_id|>Process these interactions:\n{formatted_rows}<|eot_id|>")
    ]
    
    try:
        result = llm.invoke(messages)
        content = result.content if hasattr(result, 'content') else str(result)
        
        try:
            return json.loads(content)
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {str(e)}")
            print("Raw content:", content)
            return None
            
    except Exception as e:
        print(f"Processing error: {str(e)}")
        return None

def process_transcripts_in_batches(df, llm, batch_size=5):
    """Process transcripts in batches and return results"""
    all_results = []
    
    # Calculate number of batches
    n_batches = (len(df) + batch_size - 1) // batch_size
    
    # Process each batch
    for batch_idx in tqdm(range(n_batches)):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(df))
        
        # Get batch of rows
        batch_rows = df.iloc[start_idx:end_idx].to_dict('records')
        
        # Process the batch
        result = process_batch(llm, batch_rows)
        
        if result:
            # Store the result
            all_results.append({
                'batch_idx': batch_idx,
                'start_idx': start_idx,
                'end_idx': end_idx - 1,
                'results': result
            })
            
            # Print the result immediately
            print(f"Batch {batch_idx} (rows {start_idx}-{end_idx-1}) processed:")
            print(json.dumps(result, indent=2))
            print("-" * 50)
        
        # Clear memory
        gc.collect()
    
    return all_results

In [ ]:
results = process_transcripts_in_batches(test_df, llm, batch_size = 5)

In [29]:
# Prompt with definitions, no labeled examples 
results = process_transcripts_in_batches(test_df, llm, batch_size = 5)

  0%|          | 1/593 [00:24<3:58:54, 24.21s/it]

Batch 0 (rows 0-4) processed:
[
  {
    "interaction_number": "1",
    "text": "Okay, now I want you to write four",
    "codes": [
      "IST"
    ],
    "speaker": "Teacher",
    "duration": "1.8200000000000005"
  },
  {
    "interaction_number": "2",
    "text": "four groups of two.",
    "codes": [
      "IST",
      "aOTR"
    ],
    "speaker": "Teacher",
    "duration": "2.5600000000000005"
  },
  {
    "interaction_number": "3",
    "text": "groups of two on your board.",
    "codes": [
      "IST",
      "aOTR"
    ],
    "speaker": "Teacher",
    "duration": "1.8599999999999992"
  },
  {
    "interaction_number": "4",
    "text": "Remember, the first number tells you how many groups. The second number tells you how many in each group. So how many groups do I have to say?",
    "codes": [
      "IST",
      "aOTR"
    ],
    "speaker": "Teacher",
    "duration": "9.099999999999998"
  },
  {
    "interaction_number": "5",
    "text": "How many in each? Where are my four?",
    "

  0%|          | 2/593 [00:42<3:21:40, 20.47s/it]

Batch 1 (rows 5-9) processed:
[
  {
    "interaction_number": "1",
    "text": "Yeah, some have two.",
    "codes": [
      "IST"
    ],
    "speaker": "Teacher",
    "duration": "1.68"
  },
  {
    "interaction_number": "2",
    "text": "Write it on your board and see.",
    "codes": [
      "IST",
      "aOTR"
    ],
    "speaker": "Teacher",
    "duration": "1.18"
  },
  {
    "interaction_number": "3",
    "text": "Write it on your board and see.",
    "codes": [
      "IST",
      "aOTR"
    ],
    "speaker": "Teacher",
    "duration": "1.46"
  },
  {
    "interaction_number": "4",
    "text": "your skin, Dorothy. You",
    "codes": [
      "ST"
    ],
    "speaker": "Teacher",
    "duration": "1.26"
  },
  {
    "interaction_number": "5",
    "text": "have four groups of two.",
    "codes": [
      "IST",
      "aAFF"
    ],
    "speaker": "Teacher",
    "duration": "2.58"
  }
]
--------------------------------------------------


  1%|          | 3/593 [01:05<3:36:00, 21.97s/it]

...

Batch 590 (rows 2950-2954) processed:
[
  {
    "interaction_number": "1",
    "text": "you know how to enter it correctly?",
    "codes": [
      "IST",
      "aOTR"
    ],
    "speaker": "Teacher",
    "duration": "1.5"
  },
  {
    "interaction_number": "2",
    "text": "Alright. Team 1, Team 2, Team 3, and Team 4, you may do the oh yeah for winning.",
    "codes": [
      "IST",
      "GPRS"
    ],
    "speaker": "Teacher",
    "duration": "5.82000000000005"
  },
  {
    "interaction_number": "3",
    "text": "Team 1, come mark the chart.",
    "codes": [
      "IST",
      "aOTR"
    ],
    "speaker": "Teacher",
    "duration": "3.4600000000000364"
  },
  {
    "interaction_number": "4",
    "text": "The quicker you, um, not fall, the quicker we get out.",
    "codes": [
      "IST",
      "NEU"
    ],
    "speaker": "Teacher",
    "duration": "3.519999999999982"
  },
  {
    "interaction_number": "5",
    "text": "Team 2, mark the chart.",
    "codes": [
      "IST",
      "aOTR"

100%|█████████▉| 592/593 [25:24:51<00:25, 25.37s/it]

Batch 591 (rows 2955-2959) processed:
[
  {
    "interaction_number": "1",
    "text": "3, mark the chart.",
    "codes": [
      "IST"
    ],
    "speaker": "Teacher",
    "duration": "1.9600000000000364"
  },
  {
    "interaction_number": "2",
    "text": "Let's see. It looks like some people want to go out and some don't.",
    "codes": [
      "IST"
    ],
    "speaker": "Teacher",
    "duration": "3.32000000000005"
  },
  {
    "interaction_number": "3",
    "text": "Team 4.",
    "codes": [
      "IST",
      "REP"
    ],
    "speaker": "Teacher",
    "duration": "2.67999999999995"
  },
  {
    "interaction_number": "4",
    "text": "4. Oh",
    "codes": [
      "NEU"
    ],
    "speaker": "Teacher",
    "duration": "1.1200000000000043"
  },
  {
    "interaction_number": "5",
    "text": "she doesn't want to. Oh",
    "codes": [
      "IST",
      "REP"
    ],
    "speaker": "Teacher",
    "duration": "1.840000000000032"
  }
]
--------------------------------------------------


100%|██████████| 593/593 [25:25:14<00:00, 154.32s/it]

Batch 592 (rows 2960-2964) processed:
[
  {
    "interaction_number": "1",
    "text": "I can do that too. Watch. Isn't",
    "codes": [
      "IST",
      "NEU"
    ],
    "speaker": "Teacher",
    "duration": "4.360000000000014"
  },
  {
    "interaction_number": "2",
    "text": "Isn't that fun? Oh!",
    "codes": [
      "ST",
      "NEU"
    ],
    "speaker": "Teacher",
    "duration": "1.5600000000000591"
  },
  {
    "interaction_number": "3",
    "text": "I need everyone to put their calculator back in your box. Thank",
    "codes": [
      "IST",
      "NEU"
    ],
    "speaker": "Teacher",
    "duration": "5.420000000000073"
  },
  {
    "interaction_number": "4",
    "text": "Thank you, Montesha. You're already done.",
    "codes": [
      "BSP",
      "NEU"
    ],
    "speaker": "Teacher",
    "duration": "2.1399999999999864"
  },
  {
    "interaction_number": "5",
    "text": "Five so. You",
    "codes": [
      "IST",
      "NEU"
    ],
    "speaker": "Teacher",
    "dura

In [215]:
len(results)

591

In [213]:
test_df.shape

(2965, 4)

In [216]:
test_df = test_df[~test_df["transcript"].isnull()]

In [217]:
test_df.shape

(2885, 4)

In [218]:
import re

**Copy and pasted cell output into a txt file**

In [ ]:
def clean_progress_data(file_path):
    """
    Read a text file containing progress lines and JSON data, remove progress lines,
    and combine all JSON arrays into a single list.
    
    Args:
        file_path (str): Path to the input text file
        
    Returns:
        list: Combined list of all JSON objects
    """
    # Read the file content
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Remove all progress percentage lines
    content = re.sub(r'^\s*\d+%\|[^\n]*\n', '', content, flags=re.MULTILINE)
    
    # Remove all "Batch X processed" lines
    content = re.sub(r'^Batch \d+ \(rows \d+-\d+\) processed:\n', '', content, flags=re.MULTILINE)
    
    # Remove dashed lines
    content = re.sub(r'^-{10,}\n', '', content, flags=re.MULTILINE)
    
    # Find all JSON arrays
    arrays = re.findall(r'\[\s*\{.*?\}\s*\]', content, re.DOTALL)
    
    # Extract all objects from the arrays
    all_objects = []
    for array in arrays:
        try:
            objects = json.loads(array)
            all_objects.extend(objects)
        except json.JSONDecodeError as e:
            print(f"Error parsing array: {e}")
            continue
    
    return all_objects

In [221]:
cleaned_data = clean_progress_data('results.txt')
df = pd.DataFrame(cleaned_data)

Error parsing array: Expecting ',' delimiter: line 19 column 10 (char 1742)
Error parsing array: Invalid control character at: line 3 column 166 (char 181)


In [222]:
def clean_text(text):
    """
    Clean text by removing all extra whitespace, handling null values and non-string types.
    Repeatedly applies cleaning until no double spaces remain.
    
    Args:
        text: Input text (can be string, float, or None)
    Returns:
        str: Cleaned text with single spaces between words
    """
    # Handle null values
    if pd.isna(text):
        return ""
    
    # Convert to string if not already
    text = str(text)
    
    # Initial cleaning - replace all whitespace characters with single spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading/trailing spaces
    text = text.strip()
    
    # Double-check no multiple spaces remain
    while '  ' in text:
        text = text.replace('  ', ' ')
    
    return text

# Apply the cleaning function
test_df["transcript"] = test_df["transcript"].apply(clean_text)
df["text"] = df["text"].apply(clean_text)

In [223]:
df = df[(~(df.duplicated(subset = ["text", "duration"]))) ]

In [224]:
# Strip leading and trailing whitespaces from text column
df["text"] = df["text"].str.strip()
test_df["transcript"] = test_df["transcript"].str.strip()

In [225]:
df[~(df["text"].isin(test_df["transcript"]))]

,interaction_number,text,codes,speaker,duration,type,start_time,end_time,tag_type
13,4,nan,[],Teacher,1.019999999999996,NaN,NaN,NaN,NaN
25,1,"Four. Here's group one, group two, group three, and group four. There are my groups.","[IST, aOTR]",Teacher,3.759999999999991,NaN,NaN,NaN,NaN
26,2,"Clarissa just told me I have four groups. This number tells me there's two in each group. So what is two plus two plus two plus two, Clarissa?","[IST, aOTR]",Teacher,14.919999999999987,NaN,NaN,NaN,NaN
28,4,"Excellent. Four times two, which equals what?","[IST, aAFF]",Teacher,3.519999999999982,NaN,NaN,NaN,NaN
29,5,"Albert, that's not what we're doing with the board.",[REP],Teacher,1.8799999999999957,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3563,1,nan,[NEU],Teacher,2.400000000000034,NaN,NaN,NaN,NaN
3579,2,"Justin, you are once again my best friend. That is three marks. You get one more.","[REP, GPRS]",Teacher,18.3599999999999,NaN,NaN,NaN,NaN
3580,3,"Tino, what number do you want to do? And Justin, why'd you get that?","[sOTR, REP]",Teacher,18.3599999999999,NaN,NaN,NaN,NaN
3581,4,Why'd you get a mark? Which rule did you break?,[REP],Teacher,18.3599999999999,NaN,NaN,NaN,NaN


## Iteration Two 

* Improve prompt to include more examples
* Make sure no empty lines are being passed in 
* Strip leading and trailing white space from input
* Keep test df pre processing the same as the supervised fine tuning

In [7]:
test_prefixes =  ['051', '134', '251', '027', '129', '038', '108', '107', '252']
all_train_prefixes = list(set(prefixes) - set(test_prefixes))
train_prefixes, val_prefixes = train_test_split(all_train_prefixes, test_size=0.10, random_state=99)

test_keys = [key for key in keys if key[:3] in test_prefixes]

test_dfs = [aligned_dict[key][['duration',"speaker", "label", "transcript"]] for key in test_keys]

test_df = pd.concat(test_dfs, ignore_index=True)

test_df = test_df[test_df["speaker"] != "Student"]
test_df = test_df[test_df["label"] != "UNI"]
test_df = test_df[test_df["duration"] >0.5]

test_df['label'] = test_df['label'].apply(lambda x: ' '.join([label for label in x.split() if label not in ['UNI', 'SIL']]))
test_df = test_df[~test_df["transcript"].isnull()]

In [8]:
test_df.shape

(2885, 4)

In [136]:
def process_batch(llm, batch_rows):
    """Process a batch of transcript rows and return the results"""
    try:
        # Format each row with error handling
        formatted_rows = []
        for i, row in enumerate(batch_rows):
            try:
                # Convert duration to string if it's not already
                duration = str(row.get('duration', ''))
                # Ensure transcript is a string
                transcript = str(row.get('transcript', ''))
                # Ensure speaker is a string
                speaker = str(row.get('speaker', ''))
                
                formatted_row = (
                    f"Interaction {i+1}:\n"
                    f"Speaker: {speaker}\n"
                    f"Duration: {duration}\n"
                    f"Transcript: {transcript}"
                )
                formatted_rows.append(formatted_row)
            except Exception as e:
                print(f"Error formatting row {i}: {str(e)}")
                print(f"Row content: {row}")
                continue
        
        # Join all formatted rows
        all_rows = "\n\n".join(formatted_rows)
        
        system_prompt = '''<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an education researcher analyzing classroom practices. Label each classroom interaction using these codes, applying all relevant codes to each utterance.

CODES WITH EXAMPLES:
1. IST (Instructional Talk):
   "Today we're going to learn about fractions"
   "Let's give some other people a chance"

2. ST (Social Talk):
   "How was your weekend?"
   "Stephen, are you done with number 5 yet, or do you need more time?"

3. aOTR (Academic Opportunity to Respond):
   "What's the capital of France?"
   "Solve problem number 5 on your worksheet"

4. sOTR (Social Opportunity to Respond):
   "Who wants to be line leader today?"
   "Raise your hand if you're ready to start"

5. REP (Reprimand):
   "I'm very disappointed in your behavior"
   "Stop talking while others are working"

6. RED (Redirection):
   "Johnny, I need you to please turn around"
   "Remember to raise your hand"

7. BSP (Behavior Specific Praise):
   "Great job raising your hand before speaking!"
   "I like how quietly Sarah lined up"

8. GPRS (General Praise):
   "Good work!"
   "I love how great everyone is doing."

9. aAFF (Academic Affirmation):
   "That's correct, 2 + 2 equals 4"
   "Yes, the main character is Harry"

10. aCORR (Academic Corrective):
    "Not quite, try again"
    "The answer is kilometers, not meters"

11. NEU (Neutral):
    "Now, hmmm."
    "Not right now."

12. SV (Student Voice):
    "How much money does Jody have?" (aOTR). "45" (SV). 

OUTPUT FORMAT: [
    {
        "Interaction_number": 1,
        "Transcript": "you all did a very nice job,  most of you anyway,  packing the acorns around.  Now, Jeff has asked a very good question.",
        "Labels": ["BSP", "IST", "aOTR"],
        "Duration": 5.56
    }
]

Key Rules:
- Apply ALL relevant codes to each utterance
- A single utterance may have multiple codes
- Process each interaction separately but include all in response
- Maintain exact wording from transcripts

Return only valid JSON without explanations.<|eot_id|>'''

        user_prompt = f"<|start_header_id|>user<|end_header_id|>Process these interactions:\n{all_rows}<|eot_id|>"
        
        messages = [
            ("system", system_prompt),
            ("user", user_prompt)
        ]
        
        result = llm.invoke(messages)
        content = result.content if hasattr(result, 'content') else str(result)
        
        try:
            return json.loads(content)
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {str(e)}")
            print("Raw content:", content)
            return None
            
    except Exception as e:
        print(f"Processing error in batch: {str(e)}")
        return None

def process_transcripts_in_batches(df, llm, batch_size=5):
    """Process transcripts in batches and return results"""
    all_results = []
    
    # Calculate number of batches
    n_batches = (len(df) + batch_size - 1) // batch_size
    
    # Process each batch
    for batch_idx in tqdm(range(n_batches)):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(df))
        
        try:
            # Get batch of rows
            batch_rows = df.iloc[start_idx:end_idx].to_dict('records')
            
            # Process the batch
            result = process_batch(llm, batch_rows)
            
            if result:
                # Store the result
                all_results.append({
                    'batch_idx': batch_idx,
                    'start_idx': start_idx,
                    'end_idx': end_idx - 1,
                    'results': result
                })
                
                # Print the result immediately
                print(f"Batch {batch_idx} (rows {start_idx}-{end_idx-1}) processed:")
                print(json.dumps(result, indent=2))
                print("-" * 50)
        
        except Exception as e:
            print(f"Error processing batch {batch_idx}: {str(e)}")
            continue
            
        # Clear memory
        gc.collect()
    
    return all_results

In [137]:
# Prompt with definitions and labeled examples 
iter_2_results = process_transcripts_in_batches(test_df, llm, batch_size = 5)

  0%|          | 1/577 [00:41<6:42:40, 41.95s/it]

Batch 0 (rows 0-4) processed:
[
  {
    "Interaction_number": 1,
    "Transcript": "Okay, now I want you to write",
    "Labels": [
      "IST",
      "aOTR"
    ],
    "Duration": 1.8200000000000005
  },
  {
    "Interaction_number": 2,
    "Transcript": "four groups of two.",
    "Labels": [
      "IST",
      "aOTR"
    ],
    "Duration": 2.5600000000000005
  },
  {
    "Interaction_number": 3,
    "Transcript": "groups of two on your board.",
    "Labels": [
      "IST",
      "aOTR"
    ],
    "Duration": 1.8599999999999992
  },
  {
    "Interaction_number": 4,
    "Transcript": "Remember, the first number tells you how many groups. The second number tells you how many in each group. So how many groups do I have to say?",
    "Labels": [
      "IST",
      "aOTR"
    ],
    "Duration": 9.099999999999998
  },
  {
    "Interaction_number": 5,
    "Transcript": "How many in each? Where are my four?",
    "Labels": [
      "IST",
      "aOTR"
    ],
    "Duration": 2.2600000000000016


  0%|          | 2/577 [00:58<4:18:57, 27.02s/it]

Batch 1 (rows 5-9) processed:
[
  {
    "Interaction_number": 1,
    "Transcript": "Yeah, some have two.",
    "Labels": [
      "IST"
    ],
    "Duration": 1.68
  },
  {
    "Interaction_number": 2,
    "Transcript": "Write it on your board and see.",
    "Labels": [
      "aAFF",
      "IST"
    ],
    "Duration": 1.18
  },
  {
    "Interaction_number": 3,
    "Transcript": "Write it on your board and see.",
    "Labels": [
      "aAFF",
      "IST"
    ],
    "Duration": 1.46
  },
  {
    "Interaction_number": 4,
    "Transcript": "your skin, Dorothy. You",
    "Labels": [
      "NEU"
    ],
    "Duration": 1.26
  },
  {
    "Interaction_number": 5,
    "Transcript": "have four groups of two.",
    "Labels": [
      "aAFF",
      "IST"
    ],
    "Duration": 2.58
  }
]
--------------------------------------------------


  1%|          | 3/577 [01:22<4:04:43, 25.58s/it]

...

Batch 574 (rows 2870-2874) processed:
[
  {
    "Interaction_number": 1,
    "Transcript": "you know how to enter it correctly?",
    "Labels": [
      "aOTR"
    ],
    "Duration": 1.5
  },
  {
    "Interaction_number": 2,
    "Transcript": "Alright. Team 1, Team 2, Team 3, and Team 4, you may do the oh yeah for winning.",
    "Labels": [
      "IST",
      "GPRS"
    ],
    "Duration": 5.82
  },
  {
    "Interaction_number": 3,
    "Transcript": "Team 1, come mark the chart.",
    "Labels": [
      "IST"
    ],
    "Duration": 3.46
  },
  {
    "Interaction_number": 4,
    "Transcript": "The quicker you, um, not fall, the quicker we get out.",
    "Labels": [
      "RED"
    ],
    "Duration": 3.52
  },
  {
    "Interaction_number": 5,
    "Transcript": "Team 2, mark the chart.",
    "Labels": [
      "IST"
    ],
    "Duration": 4.6
  }
]
--------------------------------------------------


100%|█████████▉| 576/577 [7:06:34<00:22, 22.73s/it]

Batch 575 (rows 2875-2879) processed:
[
  {
    "Interaction_number": 1,
    "Transcript": "3, mark the chart.",
    "Labels": [
      "aAFF"
    ],
    "Duration": 1.96
  },
  {
    "Interaction_number": 2,
    "Transcript": "Let's see. It looks like some people want to go out and some don't.",
    "Labels": [
      "IST",
      "NEU"
    ],
    "Duration": 3.32
  },
  {
    "Interaction_number": 3,
    "Transcript": "Team 4.",
    "Labels": [
      "IST",
      "NEU"
    ],
    "Duration": 2.68
  },
  {
    "Interaction_number": 4,
    "Transcript": "4. Oh",
    "Labels": [
      "IST",
      "NEU"
    ],
    "Duration": 1.12
  },
  {
    "Interaction_number": 5,
    "Transcript": "she doesn't want to. Oh",
    "Labels": [
      "IST",
      "NEU"
    ],
    "Duration": 1.84
  }
]
--------------------------------------------------


100%|██████████| 577/577 [7:06:52<00:00, 44.39s/it]

Batch 576 (rows 2880-2884) processed:
[
  {
    "Interaction_number": 1,
    "Transcript": "I can do that too. Watch. Isn't",
    "Labels": [
      "NEU"
    ],
    "Duration": 4.36
  },
  {
    "Interaction_number": 2,
    "Transcript": "Isn't that fun? Oh!",
    "Labels": [
      "ST",
      "NEU"
    ],
    "Duration": 1.56
  },
  {
    "Interaction_number": 3,
    "Transcript": "I need everyone to put your calculator back in your box. Thank",
    "Labels": [
      "IST",
      "GPRS"
    ],
    "Duration": 5.42
  },
  {
    "Interaction_number": 4,
    "Transcript": "Thank you, Montesha. You're already done.",
    "Labels": [
      "BSP",
      "REP"
    ],
    "Duration": 2.14
  },
  {
    "Interaction_number": 5,
    "Transcript": "Five so. You",
    "Labels": [
      "NEU"
    ],
    "Duration": 3.76
  }
]
--------------------------------------------------


In [11]:
len(iter_2_results)

575

In [12]:
flattened_results = []
for batch_result in iter_2_results:
    for result in batch_result['results']:
        flattened_results.append(result)

In [13]:
iter_2_results_df = pd.DataFrame(flattened_results)

In [14]:
len(iter_2_results)

575

In [15]:
import re

In [16]:
def clean_text(text):
    # Handle null values
    if pd.isna(text):
        return ""
    
    # Convert to string if not already
    text = str(text)
    
    # Initial cleaning - replace all whitespace characters with single spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading/trailing spaces
    text = text.strip()
    
    # Double-check no multiple spaces remain
    while '  ' in text:
        text = text.replace('  ', ' ')
    
    # Replace y' all with y'all
    text = re.sub(r"\s*([^\w\s.!?])\s*", r"\1", text)

    # Remove punctuation at the end of the text
    text = re.sub(r'[.,!?]+$','', text)
    
    return text.strip()

# Apply the cleaning function
test_df["transcript"] = test_df["transcript"].apply(clean_text)
iter_2_results_df["Transcript"] = iter_2_results_df["Transcript"].apply(clean_text)

test_df["duration"] = test_df["duration"].round(2)
iter_2_results_df["Duration"] = iter_2_results_df["Duration"].round(2)

In [17]:
merged_df = pd.merge(test_df,iter_2_results_df,  
                    left_on=["transcript", "duration"],  # column name in test_df
                    right_on=["Transcript", "Duration"], # column name in iter_2_results_df
                    how="inner")

In [18]:
merged_df.shape

(2770, 10)

In [19]:
# Set null values to epmty list
merged_df.loc[merged_df["Labels"].isnull(), "Labels"] = ""

In [20]:
test_df.shape

(2885, 4)

In [21]:
# Anti - join to get the unmatched rows
unmatched_test_df = test_df[~test_df["transcript"].isin(merged_df["transcript"])]
unmmatched_iter_2_results_df = iter_2_results_df[~iter_2_results_df["Transcript"].isin(merged_df["transcript"])]

In [22]:
iter_2_results_df[iter_2_results_df["Transcript"].str.contains("got twenty-three")]

,Interaction_number,Transcript,Labels,Duration,Analysis,Tags
37,3,"got twenty-three,though. All",[NEU],1.36,NaN,NaN


In [23]:
test_df[test_df["transcript"].str.contains("got twenty-three.")]

,duration,speaker,label,transcript
81,1.36,Teacher,IST,"got twenty-three,though. All"


In [24]:
unmmatched_iter_2_results_df 

,Interaction_number,Transcript,Labels,Duration,Analysis,Tags
0,1,"Okay,now I want you to write","[IST, aOTR]",1.82,NaN,NaN
114,5,"What do you do next,Clarissa",[aOTR],3.26,NaN,NaN
176,2,around. There's an example of wiggling. Very g...,"[IST, BSP]",5.64,NaN,NaN
216,2,All right. So we've got only three people rais...,"[IST, aOTR]",10.40,NaN,NaN
270,1,Three checks is all you need to lose it. All r...,[REP],5.40,NaN,NaN
...,...,...,...,...,...,...
2700,3,Thank you. Fine little Valentine for having a ...,"[BSP, IST]",7.04,NaN,NaN
2712,5,"open up. I mean,doors,so you have to look and ...","[IST, REP]",6.94,NaN,NaN
2753,1,And Calvin. And Hermon. So we should actually ...,"[IST, aAFF]",6.96,NaN,NaN
2849,2,"Justin,you are once again my best friend. That...","[BSP, REP, sOTR]",18.36,NaN,NaN


In [25]:
from fuzzywuzzy import process, fuzz

In [26]:
# Function to find best match for each row
def find_best_match(row, comparison_df, threshold=80):
    """
    Finds the best match for a row in the comparison DataFrame based on text similarity.

    Args:
        row (pd.Series): A row from the source DataFrame.
        comparison_df (pd.DataFrame): The target DataFrame to compare against.
        threshold (int): Minimum similarity score to consider a match.

    Returns:
        tuple: Best match text and its similarity score, or (None, 0) if no match above threshold.
    """
    text_to_match = row['Transcript']
    best_match, score, _ = process.extractOne(text_to_match, comparison_df['transcript'], scorer=fuzz.ratio)  # Specify a scorer
    if score >= threshold:
        return best_match, score
    return None, 0

# Apply the function to find matches
unmmatched_iter_2_results_df['best_match'], unmmatched_iter_2_results_df['similarity_score'] = zip(
    *unmmatched_iter_2_results_df.apply(find_best_match, axis=1, comparison_df=unmatched_test_df)
)

/var/folders/nj/8p2ng_756r79w77j9tf_48pc0000gn/T/ipykernel_32179/2905755444.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmmatched_iter_2_results_df['best_match'], unmmatched_iter_2_results_df['similarity_score'] = zip(
/var/folders/nj/8p2ng_756r79w77j9tf_48pc0000gn/T/ipykernel_32179/2905755444.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmmatched_iter_2_results_df['best_match'], unmmatched_iter_2_results_df['similarity_score'] = zip(


In [27]:
matched_df = unmmatched_iter_2_results_df[unmmatched_iter_2_results_df['similarity_score'] > 80]

In [28]:
close_match_df = pd.merge(test_df, matched_df ,  
                    left_on=["transcript", "duration"],  # column name in test_df
                    right_on=["best_match", "Duration"], # column name in iter_2_results_df
                    how="inner")

In [29]:
close_match_df.columns

Index(['duration', 'speaker', 'label', 'transcript', 'Interaction_number',
       'Transcript', 'Labels', 'Duration', 'Analysis', 'Tags', 'best_match',
       'similarity_score'],
      dtype='object')

In [30]:
# Select 'duration', 'speaker', 'label', 'transcript', 'Transcript', 'Labels', 'Duration' from close_match_df
close_match_df = close_match_df[['duration', 'speaker', 'label', 'transcript', 'Transcript', 'Labels', 'Duration']]

# Select 'duration', 'speaker', 'label', 'transcript', 'Transcript', 'Labels', 'Duration' from merged_df
merged_df = merged_df[['duration', 'speaker', 'label', 'transcript', 'Transcript', 'Labels', 'Duration']]

In [31]:
# Concatenate merged_df and close_match_df
final_df = pd.concat([merged_df, close_match_df], ignore_index=True)

In [32]:
final_df.shape

(2862, 7)

In [610]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

def preprocess_labels(label_series):
    """
    Preprocess labels into consistent list format
    """
    processed_labels = []
    for item in label_series:
        if isinstance(item, str):
            # Split space-separated labels
            labels = item.split()
            # Remove duplicates while preserving order
            labels = list(dict.fromkeys(labels))
            processed_labels.append(labels)
        else:
            processed_labels.append([item])
    return processed_labels

def evaluate_multilabel_performance(df, single_label_col='label', multi_label_col='Labels'):
    """
    Evaluate multi-label classification performance
    """
    df = df.copy()
    
    # Process predicted labels (space-separated strings)
    y_pred_lists = preprocess_labels(df[single_label_col])
    
    # True labels are already in list format
    y_true_lists = df[multi_label_col].tolist()
    
    # Initialize and fit MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    
    # Transform both predictions and true labels
    y_true = mlb.fit_transform(y_true_lists)
    y_pred = mlb.transform(y_pred_lists)
    
    # Calculate metrics
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    hamming_loss = np.mean(y_true != y_pred)
    
    # Calculate per-label metrics
    label_wise_f1 = {}
    for i, label in enumerate(mlb.classes_):
        label_f1 = f1_score(y_true[:, i], y_pred[:, i])
        label_wise_f1[label] = label_f1
    
    # Print results
    print("\nFinal Test Results:")
    print(f"Test Loss: {hamming_loss:.4f}")
    print(f"Test Macro F1: {macro_f1:.4f}")
    print(f"Test Micro F1: {micro_f1:.4f}\n")
    
    # Sort labels by F1 score for clearer presentation
    sorted_labels = sorted(label_wise_f1.items(), key=lambda x: x[1], reverse=True)
    for label, f1 in sorted_labels:
        print(f"{label}: F1 = {f1:.4f}")
    
    return {
        'hamming_loss': hamming_loss,
        'macro_f1': macro_f1,
        'micro_f1': micro_f1,
        'label_wise_f1': label_wise_f1,
        'classes': mlb.classes_
    }

In [611]:
final_df_copy = final_df.copy()

In [612]:
# Process true labels (space-separated strings)
y_true_lists = preprocess_labels(final_df_copy ["label"])

# Predicted labels are already in list format
y_pred_lists = final_df_copy ["Labels"].tolist()


In [613]:
# Get elements of y_pred_lists that are not a list or set 
non_list_preds = [pred for pred in y_pred_lists if not isinstance(pred, list)]

In [616]:
y_pred_lists = [label if isinstance(label, list) else [] for label in y_pred_lists]
y_true_lists = [label if isinstance(label, list) else [] for label in y_true_lists]

In [617]:
assert all(isinstance(labels, (list, set)) for labels in y_true_lists), "y_true_lists must be lists of labels"
assert all(isinstance(labels, (list, set)) for labels in y_pred_lists), "y_pred_lists must be lists of labels"

In [618]:
# Initialize and fit MultiLabelBinarizer
mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(y_true_lists)
y_pred = mlb.transform(y_pred_lists)

# Ensure shapes match
assert y_true.shape == y_pred.shape, "Shapes of y_true and y_pred do not match"

# Calculate metrics
macro_f1 = f1_score(y_true, y_pred, average='macro')
micro_f1 = f1_score(y_true, y_pred, average='micro')
hamming_loss = np.mean(y_true != y_pred)


for i, concept in enumerate(mlb.classes_):
    concept_f1 = f1_score(y_true[:, i], y_pred[:, i])
    precision = precision_score(y_true[:, i], y_pred[:, i])
    recall = recall_score(y_true[:, i], y_pred[:, i])
    print(f"{concept:25} - F1: {concept_f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

BSP                       - F1: 0.2817, Precision: 0.1709, Recall: 0.8000
GPRS                      - F1: 0.1615, Precision: 0.2100, Recall: 0.1313
IST                       - F1: 0.6859, Precision: 0.6069, Recall: 0.7887
NEU                       - F1: 0.1873, Precision: 0.1165, Recall: 0.4774
RED                       - F1: 0.2679, Precision: 0.1891, Recall: 0.4592
REP                       - F1: 0.3068, Precision: 0.2070, Recall: 0.5923
ST                        - F1: 0.2025, Precision: 0.4040, Recall: 0.1351
SV                        - F1: 0.0768, Precision: 0.8105, Recall: 0.0403
aAFF                      - F1: 0.3137, Precision: 0.3520, Recall: 0.2830
aCORR                     - F1: 0.2639, Precision: 0.5588, Recall: 0.1727
aOTR                      - F1: 0.7018, Precision: 0.7927, Recall: 0.6295
sOTR                      - F1: 0.1637, Precision: 0.1850, Recall: 0.1468


/Users/isabelarvelo/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['B', 'I', 'R'] will be ignored
  warnings.warn(


In [619]:
macro_f1 = f1_score(y_true, y_pred, average='macro')
micro_f1 = f1_score(y_true, y_pred,  average='micro')

print(f'Validation Macro F1: {macro_f1:.4f}')
print(f'Validation Micro F1: {micro_f1:.4f}')
    

Validation Macro F1: 0.3011
Validation Micro F1: 0.4179


In [621]:
label_to_concept = {
    'IST': 'Teacher_Talk',
    'ST': 'Teacher_Talk',
    'aOTR': 'Opportunity_to_Respond',
    'sOTR': 'Opportunity_to_Respond',
    'REP': 'Corrective_Behavioral_Feedback',
    'RED': 'Corrective_Behavioral_Feedback',
    'GPRS': 'Praise',
    'BSP': 'Praise',
    'aAFF': 'Academic_Feedback',
    'aCORR': 'Academic_Feedback',
    'SV': 'Other',
    'NEU': 'Other'
}

# Function to map detailed labels to broader concepts
# Function to map detailed labels to broader concepts
def map_to_broader_concepts(label_string):
    """
    Maps specific labels in a space-separated string to broader concepts.

    Args:
        label_string (str): A space-separated string of specific labels.

    Returns:
        str: A space-separated string of broader concepts, with duplicates removed.
    """
    specific_labels = label_string.split()
    broader_concepts = set(
        label_to_concept.get(label, 'Unknown') for label in specific_labels
    )  # Using `get` to handle missing keys
    return ' '.join(broader_concepts)


In [626]:
# Function to map detailed labels to broader concepts
def map_to_broader_concepts(label_input):
    """
    Maps specific labels to broader concepts.

    Args:
        label_input (str | list): A space-separated string or a list of specific labels.

    Returns:
        str: A space-separated string of broader concepts, with duplicates removed.
    """
    # If input is a string, split it into a list
    if isinstance(label_input, str):
        specific_labels = label_input.split()
    elif isinstance(label_input, list):
        specific_labels = label_input
    else:
        raise ValueError("Input must be a string or list")

    # Map to broader concepts
    broader_concepts = set(
        label_to_concept.get(label, 'Unknown') for label in specific_labels
    )  # Using `get` to handle missing keys
    return ' '.join(broader_concepts)


# Apply function to DataFrame columns
final_df['true_broader_label'] = final_df['label'].apply(map_to_broader_concepts)
final_df['predicted_broader_label'] = final_df['Labels'].apply(map_to_broader_concepts)

In [627]:
# Process true labels (space-separated strings)
y_true_lists = preprocess_labels(final_df["true_broader_label"])

In [628]:
# Predicted labels are already in list format
y_pred_lists = preprocess_labels(final_df['predicted_broader_label'])

In [629]:
y_pred_lists = [label if isinstance(label, list) else [] for label in y_pred_lists]
y_true_lists = [label if isinstance(label, list) else [] for label in y_true_lists]

In [630]:
# Initialize and fit MultiLabelBinarizer
mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(y_true_lists)
y_pred = mlb.transform(y_pred_lists)

# Ensure shapes match
assert y_true.shape == y_pred.shape, "Shapes of y_true and y_pred do not match"

# Calculate metrics
macro_f1 = f1_score(y_true, y_pred, average='macro')
micro_f1 = f1_score(y_true, y_pred, average='micro')
hamming_loss = np.mean(y_true != y_pred)

for i, concept in enumerate(mlb.classes_):
    concept_f1 = f1_score(y_true[:, i], y_pred[:, i])
    precision = precision_score(y_true[:, i], y_pred[:, i])
    recall = recall_score(y_true[:, i], y_pred[:, i])
    print(f"{concept:25} - F1: {concept_f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

Academic_Feedback         - F1: 0.3396, Precision: 0.4245, Recall: 0.2830
Corrective_Behavioral_Feedback - F1: 0.3895, Precision: 0.2781, Recall: 0.6498
Opportunity_to_Respond    - F1: 0.7215, Precision: 0.8042, Recall: 0.6542
Other                     - F1: 0.4039, Precision: 0.7466, Recall: 0.2768
Praise                    - F1: 0.4915, Precision: 0.3664, Recall: 0.7465
Teacher_Talk              - F1: 0.7440, Precision: 0.7219, Recall: 0.7675


/Users/isabelarvelo/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['Unknown'] will be ignored
  warnings.warn(


In [631]:
macro_f1 = f1_score(y_true, y_pred, average='macro')
micro_f1 = f1_score(y_true, y_pred,  average='micro')

print(f'Validation Macro F1: {macro_f1:.4f}')
print(f'Validation Micro F1: {micro_f1:.4f}')

Validation Macro F1: 0.5150
Validation Micro F1: 0.5907


In [632]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, classification_report, precision_score, recall_score

In [633]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

def preprocess_labels(label_series):
    """
    Preprocess labels into consistent list format
    """
    processed_labels = []
    for item in label_series:
        if isinstance(item, str):
            # Split space-separated labels
            labels = item.split()
            # Remove duplicates while preserving order
            labels = list(dict.fromkeys(labels))
            processed_labels.append(labels)
        else:
            processed_labels.append([item])
    return processed_labels
